In [1]:
pip install medmnist torch torchvision matplotlib seaborn scikit-learn umap-learn

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import pandas as pd

# Load the npz file directly
data = np.load('pathmnist.npz')

# Combine all splits
all_images = np.concatenate([
    data['train_images'],
    data['val_images'],
    data['test_images']
], axis=0)

all_labels = np.concatenate([
    data['train_labels'],
    data['val_labels'],
    data['test_labels']
], axis=0).flatten()

# Flatten each image from (28, 28, 3) → 1D vector
flat_images = all_images.reshape((all_images.shape[0], -1))

# Create DataFrame: first column is label, rest are pixels
df_full = pd.DataFrame(flat_images)
df_full.insert(0, "label", all_labels)

# Preview
print(df_full.head())

# Optional: Save to CSV
df_full.to_csv("pathmnist_full_dataset.csv", index=False)

   label      0      1      2      3      4      5      6      7      8  ...  \
0    0.0  220.0  208.0  227.0  219.0  207.0  227.0  220.0  207.0  227.0  ...   
1    4.0  126.0   59.0  119.0  142.0   84.0  139.0  174.0  127.0  176.0  ...   
2    7.0  223.0  168.0  204.0  223.0  166.0  203.0  218.0  170.0  205.0  ...   
3    5.0  216.0  135.0  183.0  216.0  127.0  178.0  213.0  125.0  175.0  ...   
4    5.0  204.0  124.0  177.0  204.0  116.0  170.0  211.0  149.0  190.0  ...   

    2342   2343   2344   2345   2346   2347   2348   2349   2350   2351  
0  227.0  220.0  208.0  227.0  221.0  208.0  227.0  222.0  209.0  228.0  
1  211.0  185.0  150.0  193.0  168.0  126.0  177.0  177.0  137.0  184.0  
2  178.0  220.0  163.0  200.0  223.0  159.0  199.0  223.0  155.0  196.0  
3  188.0  212.0  130.0  178.0  214.0  129.0  178.0  218.0  133.0  181.0  
4  180.0  194.0  124.0  176.0  217.0  170.0  205.0  229.0  206.0  225.0  

[5 rows x 2353 columns]


In [4]:
# Step 1: Define the mapping from label numbers to medical class names
label_map = {
    0: 'ADI',
    1: 'BACK',
    2: 'DEB',
    3: 'LYM',
    4: 'MUC',
    5: 'MUS',
    6: 'NORM',
    7: 'STR',
    8: 'TUM'
}

# Step 2: Apply stratified sampling (4500 rows per class)
df_sampled = df_full.groupby('label').apply(lambda x: x.sample(n=4500, random_state=42)).reset_index(drop=True)

# Step 3: Replace numeric labels with string labels using the mapping
df_sampled['label'] = df_sampled['label'].map(label_map)

# Save to CSV
df_sampled.to_csv("pathmnist_stratified_4500_per_class_labeled.csv", index=False)

# Confirm shape
print(df_sampled.shape)

/var/folders/sy/hms76h_s3393m0hc_w8sjrvh0000gn/T/ipykernel_28951/508220620.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sampled = df_full.groupby('label').apply(lambda x: x.sample(n=4500, random_state=42)).reset_index(drop=True)


(40500, 2353)


In [5]:
# Step 4: Check and drop missing values
missing_before = df_sampled.isnull().sum().sum()
df_sampled = df_sampled.dropna()
missing_after = df_sampled.isnull().sum().sum()

# Step 5: Check and drop duplicate rows
duplicates_before = df_sampled.duplicated().sum()
df_sampled = df_sampled.drop_duplicates()
duplicates_after = df_sampled.duplicated().sum()

# Save cleaned dataset again
df_sampled.to_csv("pathmnist_balanced_labeled_cleaned.csv", index=False)

# Print summary
print(f"Missing values removed: {missing_before - missing_after}")
print(f"Duplicate rows removed: {duplicates_before - duplicates_after}")
print(f"Final shape: {df_sampled.shape}")


Missing values removed: 0
Duplicate rows removed: 0
Final shape: (40500, 2353)


In [ ]:
# Step 6: One-hot encode the label column and keep the original label
if 'label' in df_sampled.columns:
    df_encoded = pd.get_dummies(df_sampled, columns=['label'])
    df_encoded['label'] = df_sampled['label']  # reattach the original label
else:
    print("Error: 'label' column not found in df_sampled.")

# Save the final one-hot encoded dataset
df_encoded.to_csv("pathmnist_preprocessed_final.csv", index=False)

# Confirmation message
print(f"\n✅ Final one-hot encoded dataset saved as 'pathmnist_preprocessed_final.csv'.")
print(f"Final DataFrame shape: {df_encoded.shape}")


✅ Final one-hot encoded dataset saved as 'pathmnist_preprocessed_final.csv'.
Final DataFrame shape: (40500, 2362)
